In [1]:
from bs4 import BeautifulSoup
from urllib.request import *
import concurrent.futures

In [2]:
links = []
links_DB = []

In [3]:
count = 0
count_error = 0
fail_url1 = []
fail_url2 = []

opener = build_opener()
opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
install_opener(opener)
all_title, all_time_pub, all_keywords, quantity_articles = [], [], [], []

In [4]:
def save_article_data(url):
    global count, count_error
    bool1 = bool2 = False
    try:
        html = urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')
        
        article_title = soup.find_all('h1', class_="c-article-title")
        all_title.append(article_title[0].text)
        
        time_published = soup.find_all('time')
        all_time_pub.append(time_published[0].text)
        
        keywords = soup.find_all('span', itemprop="about")
        all_keywords.append([keywords[i].text for i in range(len(keywords))])
        
        links_DB.append(url)
        count += 1 
        if count % 1000 == 0:
            print(count, end=' ')
    except Exception as exc:
        count_error += 1
        fail_url1.append(url)
        print('error -', exc, '  url: ', url)
        
def get_num_articles(arr):
    count = 0
    for url in arr:
        html = urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')
        article_title = soup.find_all('strong')
        quantity_articles.append(article_title[0].text)

        count += 1 #         
        if count % 10 == 0:
                print(count)

In [5]:
def save_link_article(url):
    global count, count_error
    bool1 = bool2 = False
    try:
        html = urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')
        items = soup.find_all('a', class_='title')# class_="c-article-title")
       # print(items)
        for item in items:
            links.append('https://link.springer.com/' + item['href'])
        count += 1 #         for item in items:#             print(count, item['src'])
        if count % 1000 == 0:
            print(count, end=' ')
    except Exception as exc:
        print('error -', exc, '  url: ', url)

In [6]:
%%time

arr = ['https://link.springer.com/search/page/1?facet-start-year=' + str(i) + '&facet-end-year=' + str(i) + '&previous-start-year=1966&previous-end-year=2021&date-facet-mode=in&sortOrder=newestFirst&showAll=true&facet-discipline=%22Computer+Science%22&facet-content-type=%22Article%22' for i in range(1966, 2022)]

get_num_articles(arr)

10
20
30
40
50
Wall time: 23.6 s


In [7]:
quantity_articles = [quantity_articles[i].replace(',', '') for i in range(len(quantity_articles))]

URLs = []

for i, row in enumerate(arr):
    k = int(quantity_articles[i])
    for j in range(k // 20 + 1 if k % 20 != 0 else k // 20):
        URLs.append(row[:38] + str(j + 1) + row[39:])
    
URLs.reverse()

In [8]:
%%time

with concurrent.futures.ThreadPoolExecutor(max_workers=64) as executor:
    future_to_url = {executor.submit(save_link_article, url): 
                            url for url in URLs}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            future.result()
        except Exception as exc:
            print("%r generated an exception: %s" % (url, exc))
                
count, count_error, len(fail_url1), len(fail_url2)



1000 2000 3000 4000 5000 6000 7000 Wall time: 5min 31s


(7679, 0, 0, 0)

In [9]:
len(links), links

(153030,
 ['https://link.springer.com//article/10.1007/s10462-020-09851-4',
  'https://link.springer.com//article/10.1134/S0361768821010060',
  'https://link.springer.com//article/10.1007/s10623-020-00813-z',
  'https://link.springer.com//article/10.1007/s11623-020-1388-8',
  'https://link.springer.com//article/10.1007/s10639-020-10312-2',
  'https://link.springer.com//article/10.1007/s11227-020-03272-4',
  'https://link.springer.com//article/10.1007/s10639-020-10340-y',
  'https://link.springer.com//article/10.1007/s00521-020-04984-7',
  'https://link.springer.com//article/10.1007/s10489-020-01805-1',
  'https://link.springer.com//article/10.1007/s00453-020-00758-8',
  'https://link.springer.com//article/10.1007/s10489-020-01706-3',
  'https://link.springer.com//article/10.1007/s11042-020-09970-8',
  'https://link.springer.com//article/10.1007/s11623-020-1387-9',
  'https://link.springer.com//article/10.1007/s00521-020-05057-5',
  'https://link.springer.com//article/10.1007/s10639-020

In [10]:
%%time

#arr = ['https://link.springer.com/search/page/1?facet-start-year=' + str(i) + '&facet-end-year=' + str(i) + '&previous-start-year=1966&previous-end-year=2021&date-facet-mode=in&sortOrder=newestFirst&showAll=true&facet-discipline=%22Computer+Science%22&facet-content-type=%22Article%22' for i in range(1966, 2022)]

with concurrent.futures.ThreadPoolExecutor(max_workers=128) as executor:
    future_to_url = {executor.submit(save_article_data, url): 
                            url for url in links}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            future.result()
        except Exception as exc:
            print("%r generated an exception: %s" % (url, exc))
                
count, count_error, len(fail_url1), len(fail_url2)



error - [WinError 10054] Удаленный хост принудительно разорвал существующее подключение   url:  https://link.springer.com//article/10.1007/s00453-020-00760-0
error - [WinError 10054] Удаленный хост принудительно разорвал существующее подключение   url:  https://link.springer.com//article/10.1007/s00366-019-00811-3
error - [WinError 10054] Удаленный хост принудительно разорвал существующее подключение   url:  https://link.springer.com//article/10.1007/s10639-020-10264-7
8000 error - [WinError 10054] Удаленный хост принудительно разорвал существующее подключение   url:  https://link.springer.com//article/10.1007/s00366-019-00822-0
error - [WinError 10054] Удаленный хост принудительно разорвал существующее подключение   url:  https://link.springer.com//article/10.1007/s11042-020-09783-9
error - [WinError 10054] Удаленный хост принудительно разорвал существующее подключение   url:  https://link.springer.com//article/10.1007/s00366-020-01229-y
error - [WinError 10054] Удаленный хост принуди

(159747, 962, 962, 0)

In [11]:
all_title

['\n   Das IT-Sicherheitsgesetz 2.0\n  ',
 '\n   Vertragliche Absicherung internationaler Datentransfers\n  ',
 '\n   Benutzerfreundlicher Datenschutz in Cloud-basierten Office-Paketen\n  ',
 'An Algorithm for Solving a Family of Fourth-Degree Diophantine Equations that Satisfy Runge’s Condition',
 'Risk-free WHO grading of astrocytoma using convolutional neural networks from MRI images',
 'Domain knowledge graph-based research progress of knowledge representation',
 'Neural network-based indoor localization system with enhanced virtual access points',
 'An enhanced cost-aware mapping algorithm based on improved shuffled frog leaping in network on chips',
 'Correction to: On the Expressive Power of Linear Algebra on Graphs',
 'On Structural Parameterizations of the Bounded-Degree Vertex Deletion Problem',
 'A systematic literature review of multicriteria recommender systems',
 'Collaborative writing at work: Peer feedback in a blended learning environment',
 'Decision-making method bas

In [12]:
all_time_pub

['14 January 2021',
 '14 January 2021',
 '14 January 2021',
 '23 February 2021',
 '28 September 2020',
 '21 June 2020',
 '22 April 2020',
 '20 April 2020',
 '11 January 2021',
 '19 August 2020',
 '09 June 2020',
 '25 August 2020',
 '12 August 2020',
 '08 August 2020',
 '29 October 2020',
 '01 October 2020',
 '07 October 2020',
 '27 August 2020',
 '25 January 2021',
 '17 December 2020',
 '08 April 2021',
 '29 October 2020',
 '14 August 2020',
 '07 September 2020',
 '17 August 2020',
 '25 August 2020',
 '13 May 2021',
 '08 July 2020',
 '01 October 2020',
 '11 September 2020',
 '22 September 2020',
 '10 September 2020',
 '19 April 2021',
 '19 September 2020',
 '21 May 2020',
 '19 August 2020',
 '13 May 2021',
 '18 September 2020',
 '19 September 2020',
 '21 March 2020',
 '17 August 2020',
 '04 March 2020',
 '07 July 2020',
 '18 September 2020',
 '07 April 2020',
 '17 October 2020',
 '16 May 2020',
 '21 September 2020',
 '27 July 2020',
 '23 August 2019',
 '29 September 2020',
 '17 July 20

In [13]:
all_keywords

[[],
 [],
 [],
 [],
 ['WHO grading', 'Astrocytoma grading', 'Convolutional neural networks'],
 ['Domain knowledge graph',
  'Knowledge representation',
  'Entity',
  'Relationship',
  'Property'],
 ['Indoor localization',
  'Neural network',
  'Virtual access points',
  'WiFi',
  'Kriging algorithm',
  'RSSI'],
 ['Network on chip (NoC)',
  'System on chip (SoC)',
  'Application mapping',
  'Improved shuffled frog leaping algorithm (ISFL)',
  'Communication cost'],
 [],
 ['Bounded-degree vertex deletion',
  'Feedback vertex set',
  'Parameterized algorithms',
  'Treecut width'],
 ['Recommender system',
  'Multicriteria recommendation',
  'Systematic literature review',
  'Survey'],
 ['Collaborative writing',
  'Reflective practices',
  'Peer feedback, blended learning environment',
  'Assessment of learning'],
 ['Multi-attribute decision-making. Refined single-valued neutrosophic sets (RSVNSs). Evaluation based on distance from average solution (EDAS) method. Neutrosophic entropy. Typho

In [15]:
all_keywords_join = []

for i in range(len(all_keywords)):
    all_keywords_join.append(', '.join(all_keywords[i]))

In [16]:
all_keywords_join

['',
 '',
 '',
 '',
 'WHO grading, Astrocytoma grading, Convolutional neural networks',
 'Domain knowledge graph, Knowledge representation, Entity, Relationship, Property',
 'Indoor localization, Neural network, Virtual access points, WiFi, Kriging algorithm, RSSI',
 'Network on chip (NoC), System on chip (SoC), Application mapping, Improved shuffled frog leaping algorithm (ISFL), Communication cost',
 '',
 'Bounded-degree vertex deletion, Feedback vertex set, Parameterized algorithms, Treecut width',
 'Recommender system, Multicriteria recommendation, Systematic literature review, Survey',
 'Collaborative writing, Reflective practices, Peer feedback, blended learning environment, Assessment of learning',
 'Multi-attribute decision-making. Refined single-valued neutrosophic sets (RSVNSs). Evaluation based on distance from average solution (EDAS) method. Neutrosophic entropy. Typhoon disaster assessment',
 'K-12 education, End-user computing, High-mathability computer problem-solving, S

In [17]:
for i in range(len(all_keywords_join)):
    if all_keywords_join[i] == '':
        all_keywords_join[i] = 'NONE'

In [18]:
import sqlite3 as sq

with sq.connect("E:\\Frontir.db") as con:
    cur = con.cursor()
    cur.execute("""CREATE TABLE IF NOT EXISTS articles (
    name TEXT,
    data TEXT,
    keywords TEXT,
    url TEXT)
    """)
    for i in range(len(all_title)):
        cur.execute("""INSERT INTO articles(name, data, keywords, url)
           VALUES(?, ?, ?, ?)""", (all_title[i], all_time_pub[i], all_keywords_join[i], links_DB[i]))

    